## Install Packages

In [3]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("awscli==1.22.3")
install("s3fs==2022.01.0")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 KB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 20.5 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 106.7 MB/s eta 0:00:00
  Created wheel for aiobotocore: filename=aiobotocore-2.1.2-py3-none-any.whl size=58584 sha256=a3770de01eae3e1bb1259d5aaac406c1319fb2696093e3bb12a954b5d3997be0
  Stored in directory: /home/jovyan/.cache/pip/wheels/0e/9e/81/732cf36b7a7e73f82ef7793b779210f0bf94e12c13b3f2a18e
Successfully built aiobotocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.3
    Uninstalling botocore-1.23.3:
      Successfully uninstalled botocore-1.23.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.3 requires botocore==1.23.3, but you have botocore 1.23.24 which is incompatible.
tensorflow-gpu 2.8.0 requires numpy>=1.20, but you have numpy 1.18.5 which is incompatible.
kfp 1.8.11 requires typing-extensions<4,>=3.7.4; python_version < "3.9", but you have typing-extensions 4.1.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Import Libraries

In [4]:
import s3fs
import pandas as pd
import numpy as np 
import pickle
import sys
import os 
from typing import List
from typing import Optional 
import re
import pyarrow.parquet as pq
import pyarrow as pa

import xgboost  
from tensorflow.keras.models import load_model 
import warnings
warnings.filterwarnings("ignore")

2022-03-28 02:06:30.520761: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
## Test Code Block

In [9]:
fs = s3fs.S3FileSystem(anon=False, key='****', secret='****')
fs.ls("s3://zigbang-data/stage/dw_member_loyalty_inference_result/tmpdata")
fs.ls("s3://zigbang-mlops/loyalty")

['zigbang-mlops/loyalty/',
 'zigbang-mlops/loyalty/mean.npy',
 'zigbang-mlops/loyalty/model.json',
 'zigbang-mlops/loyalty/model_logit.pkl',
 'zigbang-mlops/loyalty/model_nn.h5',
 'zigbang-mlops/loyalty/model_xgb.json',
 'zigbang-mlops/loyalty/std.npy']

## Define Functions

In [10]:
AWS_KEY = "***"
AWS_SECRET = "***"

In [11]:
def timer(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        start_time = perf_counter()
        to_execute = fn(*args, **kwargs)
        end_time = perf_counter()
        execution_time = end_time - start_time
        print('{0} took {1:.8f}s to execute'.format(fn.__name__, execution_time))
        return to_execute

    return inner 

In [12]:
class BatchInput:
    base_date: str
    target_db: str

In [13]:
class LoyaltyClassifier:
    _CONST_MODEL_BUCKET_PATH = "s3://zigbang-mlops/loyalty" 
    _CONST_MODEL_XGB_PATH = "model/model_xgb.json"
    _CONST_MODEL_LR_PATH = "model/model_logit.pkl"
    _CONST_MODEL_NN_PATH = "model/model_nn.h5"
    _CONST_MODEL_MEAN_PATH = "model/mean.npy"
    _CONST_MODEL_STD_PATH = "model/std.npy"

    _CONST_BASE_S3_URI = "s3://zigbang-data/"
    _CONST_DATA_PATH = "/dw_member_loyalty_feature/"
    _CONST_OUTPUT_PATH = "/dw_member_loyalty_inference_result"  
    
    def __init__(self) -> None:
        self._s3_model_loader()
        self.models, self.mean, self.std = self._load_model() 
             
    @timer
    def batch_classify(self, input):
        df = self._s3_data_loader(input)
        feature_list = [f"f{i}" for i in range(1,19)] 
 
        user_no, X_pred, y_pred = df["user_no"], df[feature_list], np.where(df["call_cnt"]>0, 1, 0)
        
        # standardization
        X_pred -= self.mean
        X_pred /= self.std
             
        d_pred = xgboost.DMatrix(X_pred, label=y_pred)
        prob_xgb = self.models['xgb'].predict(d_pred)
        prob_lr = self.models['lr'].predict(X_pred)
        prob_nn = self.models['nn'].predict(X_pred).flatten()
         
        result = pd.concat([user_no, X_pred, pd.Series(y_pred, name="call"), \
            pd.Series((prob_xgb+prob_lr+prob_nn)/3, name="prob"), \
            pd.Series(prob_xgb, name="prob_xgb"), pd.Series(prob_lr, name="prob_lr"), \
            pd.Series(prob_nn, name="prob_nn"), df["roll_period"]], axis=1)
        # add base date on dag - task2
        #result['base_date'] = pd.to_datetime(input.base_date).strftime("%Y-%m-%d") 
                 
        return result 
 
    @timer
    def _load_model(self): 
        models = {}
        model_xgb = xgboost.Booster()
        model_xgb.load_model(self._CONST_MODEL_XGB_PATH)
        models['xgb'] = model_xgb

        model_lr = pickle.load(open(self._CONST_MODEL_LR_PATH, "rb"))
        models['lr'] = model_lr

        model_nn = load_model(self._CONST_MODEL_NN_PATH)
        models['nn'] = model_nn

        mean = np.load(self._CONST_MODEL_MEAN_PATH)
        std = np.load(self._CONST_MODEL_STD_PATH)
        return models, mean, std
        
    @timer
    def _s3_model_loader(self):
        fs = s3fs.S3FileSystem(anon=False, key=AWS_KEY, secret=AWS_SECRET) 
        fs.ls(self._CONST_MODEL_BUCKET_PATH)
        fs.download("zigbang-mlops/loyalty/model_xgb.json", self._CONST_MODEL_XGB_PATH)
        fs.download("zigbang-mlops/loyalty/model_logit.pkl", self._CONST_MODEL_LR_PATH)
        fs.download("zigbang-mlops/loyalty/model_nn.h5", self._CONST_MODEL_NN_PATH)
        fs.download("zigbang-mlops/loyalty/mean.npy", self._CONST_MODEL_MEAN_PATH)
        fs.download("zigbang-mlops/loyalty/std.npy", self._CONST_MODEL_STD_PATH)

    @timer
    def _s3_data_loader(self, input):
        fs = s3fs.S3FileSystem(anon=False, key=AWS_KEY, secret=AWS_SECRET) 
        print("target url: ", self._CONST_BASE_S3_URI+input.target_db+self._CONST_DATA_PATH)
        batch_data = fs.ls(self._CONST_BASE_S3_URI+input.target_db+self._CONST_DATA_PATH, refresh=True)
        data = ''

        try: 
            for batch in batch_data:
                if bool(re.search(input.base_date, batch)) is True:
                    uri = f's3://{batch}'
                    data = fs.ls(uri, refresh=True)[0]
            if(len(data) == 0):
                raise Exception('wrong s3 uri: ', data) 
            
        except Exception as e:
            print(e)

        bucket_uri = f's3://{data}' 
        df = pq.ParquetDataset(bucket_uri, filesystem=fs).read_pandas().to_pandas()
        
        return df 

## Batch Inference Step

In [ ]:
batch_input = BatchInput()
batch_input.base_date = "2022-03-08"
batch_input.target_db = "stage"

classifier = LoyaltyClassifier()

 
result = classifier.batch_classify(batch_input) 

_s3_model_loader took 0.86476969s to execute
_load_model took 0.21858326s to execute
target url:  s3://zigbang-data/stage/dw_member_loyalty_feature/
_s3_data_loader took 3.77606619s to execute


## Upload parquet temp table to S3

In [1]:
table = pa.Table.from_pandas(result) 
    
print("uploading tmp table...")
fs = s3fs.S3FileSystem(anon=False, key=AWS_KEY, secret=AWS_SECRET) 
# temp file clean up
if(len(fs.ls("s3://zigbang-data/"+batch_input.target_db+"/dw_member_loyalty_inference_result/tmpdata")) > 0):
    print("clean up tmp files...")
    fs.rm("s3://zigbang-data/"+batch_input.target_db+"/dw_member_loyalty_inference_result/tmpdata", "-rf")
print("writing tmp table...")
pq.write_to_dataset(table=table, \
    root_path="s3://zigbang-data/"+batch_input.target_db+"/dw_member_loyalty_inference_result/tmpdata", filesystem=fs)
print("done")

NameError: name 'pa' is not defined